<a href="https://colab.research.google.com/github/aksekhar/Signal-Strength-or-quality-model./blob/master/Signal_Strength_or_quality_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hellping Class and functions

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def mount_drive(fpath):
  try:
    from google.colab import drive
    drive.mount("/content/drive/", force_remount=True)
    google_drive_prefix = "/content/drive/My Drive"
    data_path = f"{google_drive_prefix}/{fpath}"
    print(data_path)
    return data_path
  except ModuleNotFoundError:
    data_prefix = f"{fpath}"

In [ ]:
def read_file(file_name):
  print(f"reding the file: {file_name}")
  try:
    file_type = file_name.split('.')[1]
    print(file_type)
    if file_type == 'csv':
      df = pd.read_csv(signal_file_path)
    else:
      pass
    return df
  except Exception as e:
    print("data write")
    raise e

In [ ]:
def verify_sample_data(df,records):
  if not(records):
    records = 5
  print(f"verifying first {records} records and last {records} records")
  return pd.concat([df.head(records),df.tail(records)])

In [ ]:
def get_percentage_of_missing_values(df):
  print("Checking for missing values and print percentage for each attribute")
  return (round(df.isnull().sum() / (df.isnull().count())*100))

# 1. Data import and Understanding

In [ ]:
signal_file_path = mount_drive("dataset/NN Project Data - Signal.csv")

*a. Read the ‘Signals.csv’ as DataFrame and import required libraries.*

In [ ]:
signalDf = read_file(signal_file_path)
verify_sample_data(signalDf,3)

In [ ]:
signalDf.info()

- The dataset consists of **1599 entries** (non-null values).
- There are **12 columns** in total, labeled as `Parameter 1` through `Parameter 11` and `Signal_Strength`.
- All `Parameter` columns contain **floating-point values** (float64), which suggests they could be measurements or scores of some kind.
- The `Signal_Strength` column contains **integer values** (int64), which could be a count, a rating, or some other discrete value.
- There are **no missing values** in the dataset, as indicated by the 'non-null' count for each column.


In [ ]:

signalDf.describe().T

1. **Parameter 1**: The data is centered around 8.32 with a spread of 1.74. The wide range suggests diverse data points.

2. **Parameter 2**: The average value is 0.53, indicating that most data points are close to this value. However, the maximum value of 1.58 suggests the presence of higher values as well.

3. **Parameter 3**: With a mean of 0.27 and a maximum value of 1, it appears that the data points are skewed towards the lower end.

4. **Parameter 4**: The data has a fairly large spread (std: 1.41) around the mean (2.54), indicating variability in the data points.

5. **Parameter 5**: The data is tightly clustered around the mean (0.087), as indicated by the small standard deviation (0.047).

6. **Parameter 6**: The large standard deviation (10.46) relative to the mean (15.87) suggests a wide spread in the data.

7. **Parameter 7**: The data points are spread out over a large range (from 6 to 289), indicating high variability.

8. **Parameter 8**: The data is tightly clustered around the mean (0.997), suggesting low variability.

9. **Parameter 9**: The data points are relatively close to the mean (3.31), indicating moderate variability.

10. **Parameter 10**: The data has a moderate spread (std: 0.17) around the mean (0.658), indicating some variability.

11. **Parameter 11**: The data points are fairly close to the mean (10.42), suggesting moderate variability.

12. **Signal Strength**: The average signal strength is 5.64 with a moderate spread (std: 0.81), indicating that the signal strength varies moderately across observations.

In [ ]:
get_percentage_of_missing_values(signalDf)